# Evaluating the Climate sensitivity of HR- and LR-CESM

In [ ]:
import os
import sys
import numpy as np
import xarray as xr
import cmocean
import cartopy
import scipy.stats as stats
import cartopy.crs as ccrs
import datetime
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline
mpl.rc_file('../rc_file')
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2

In [ ]:
sys.path.append("..")
from tqdm import notebook
from paths import path_results, path_prace, CESM_filename, file_ex_atm_lpd
from filters import lowpass
from constants import spy
from xr_regression import xr_linear_trend, xr_lintrend
from constants import R_earth
A_Earth = 4*np.pi*R_earth**2

In [ ]:
ds = xr.open_dataset(file_ex_atm_lpd, decode_times=False)
ds

In [ ]:
# made in GMST.ipynb
gmst_ctrl = xr.open_dataarray(f'{path_results}/GMST/GMST_monthly_ctrl.nc', decode_times=False)
gmst_rcp  = xr.open_dataarray(f'{path_results}/GMST/GMST_monthly_rcp.nc' , decode_times=False)
gmst_hq   = xr.open_dataarray(f'{path_results}/GMST/GMST_monthly_hq.nc'  , decode_times=False)
gmst_lpd  = xr.open_dataarray(f'{path_results}/GMST/GMST_monthly_lpd.nc' , decode_times=False)
gmst_lr1  = xr.open_dataarray(f'{path_results}/GMST/GMST_monthly_lr1.nc' , decode_times=False)
gmst_lq   = xr.open_dataarray(f'{path_results}/GMST/GMST_monthly_lq.nc'  , decode_times=False)
gmst_ld   = xr.open_dataarray(f'{path_results}/GMST/GMST_monthly_ld.nc'  , decode_times=False)


In [ ]:
GMST_h200  = xr.open_dataarray(f'{path_results}/GMST/GMST_monthly_ctrl.nc', decode_times=False).sel(time=slice(200*365,230*365)).mean()
GMST_l500  = xr.open_dataarray(f'{path_results}/GMST/GMST_monthly_lpd.nc' , decode_times=False).isel(time=slice(0,30*12)).mean()
GMST_l1200 = xr.open_dataarray(f'{path_results}/GMST/GMST_monthly_lpd.nc' , decode_times=False).isel(time=slice(690*12,720*12)).mean()

In [ ]:
# from `TOA.ipynb`
FTNT_rcp  = xr.open_dataarray(f'{path_prace}/TOA/TOM_rcp.nc', decode_times=False)
FTNT_lr1  = xr.open_dataarray(f'{path_prace}/TOA/TOM_lr1.nc', decode_times=False)
FTNT_hq   = xr.open_dataarray(f'{path_prace}/TOA/TOM_hq.nc' , decode_times=False)
FTNT_lq   = xr.open_dataarray(f'{path_prace}/TOA/TOM_lq.nc' , decode_times=False)
FTNT_ld   = xr.open_dataarray(f'{path_prace}/TOA/TOM_ld.nc' , decode_times=False)

## CESM response plot

## CESM ECS/TCR ; Gregory plot

In [ ]:
# transient climate sensitivity around year 2078
print((gmst_rcp.isel(time=slice(12*67,12*87)).mean('time')-GMST_h200).values)
print((gmst_lr1.isel(time=slice(12*67,12*87)).mean('time')-GMST_l500).values)


In [ ]:
lam, Delta, sq, p20 = r'$\lambda$', r'$\Delta$', r'$^2$', r'$_{20+}$'
f, ax = plt.subplots(1, 2, figsize=(6.4,3), sharey=True, constrained_layout=True)
ax[0].set_title(r'HR-CESM 4$\times$CO$_2$')
ax[0].set_xlabel(f'{Delta}GMST [K]')
ax[0].set_ylabel(f'TOA [W/m{sq}]')
ax[1].set_title(r'LR-CESM 4$\times$CO$_2$')
ax[1].set_xlabel(f'{Delta}GMST [K]')
for i in range(3):
    ax_ = [ax[0], ax[1], ax[1]][i]
    (c1,c2) = [('C0','C1'),('C2','C3'),('C0','C1')][i]
    GMST = [gmst_hq, gmst_ld, gmst_lq][i].rolling(time=12).mean()[11::12]-[GMST_h200, GMST_l500, GMST_l1200][i]
    TOA = [FTNT_hq, FTNT_ld, FTNT_lq][i]/A_Earth
    ax_.axhline(0, c='k', lw=.5)
    ax_.scatter(GMST, TOA, s=5)
    m, b = np.polyfit(GMST, TOA, 1)
    ax_.plot([0, -b/m], [b,0], c=c1, lw=.8)
#     ax_.text(.2,.9, f'{lam} = {-m:3.2f} W/m{sq}/K', transform=ax_.transAxes, c=c1)
    ax_.text(.5,[.8,.55,.8][i]+.1, f'ECS = {-b/m/[2,1,2][i]:3.2f} K', transform=ax_.transAxes, c=c1)
    print(f'{lam} = {-m:3.2f} W/m{sq}/K        ECS = {-b/m/[2,1,2][i]:3.2f} degC')
    
    m, b = np.polyfit(GMST[20:], TOA[20:], 1)
    ax_.plot([0, -b/m], [b,0], c=c2, lw=.8)
#     ax_.text(0.05,.25, f'only year 20+', transform=ax_.transAxes, c=c2)
#     ax_.text(0.05,.15, f'{lam} = {-m:3.2f} W/m{sq}/K', transform=ax_.transAxes, c=c2)
    ax_.text(0.5,[.7,.45,.7][i]+.1, f'ECS{p20} = {-b/m/[2,1,2][i]:3.2f} K', transform=ax_.transAxes, c=c2)
    print(f'{lam}_20 = {-m:3.2f} W/m{sq}/K     ECS_20 = {-b/m/[2,1,2][i]:3.2f} degC')
    print('')
    ax_.set_xlim((0,-b/m+.5))
    ax_.set_ylim((0,9))
    plt.savefig(f'{path_results}/ECS/Gregory_plots')
    plt.savefig(f'{path_results}/thesis/62_Gregory_plots.eps')

In [ ]:
plt.figure(figsize=(6.4,2))
x = .7
plt.text(x,  0, 'IPCC AR5'                , va='center')
plt.text(x, -1, 'Sherwood et al. (2020)'  , va='center')
plt.text(x, -2, 'Meehl et al. (2011)'     , va='center')
plt.text(x, -3, r'LR-CESM 4$\times$CO$_2$', va='center')
plt.text(x, -4, r'HR-CESM 4$\times$CO$_2$', va='center')
plt.text(.5,-3, 'CESM', rotation=90, va='center')
plt.plot([x-1,5],[-1.5,-1.5], c='grey', lw=.5, clip_on = False)
plt.scatter(3, 0)
plt.errorbar(3, 0, xerr=1.5)
plt.errorbar(3.25, -1, xerr=1.3/2)
plt.scatter(4.1, -2)
plt.scatter(3.5, -3)
plt.scatter(3.5, -4)
plt.yticks([])
plt.xlabel('ECS estimates  [K]')
plt.xlim((1.4,4.6))
plt.ylim((-4.5,.5))
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_visible(False)
plt.gca().xaxis.set_label_position('top')
plt.gca().tick_params(axis="x", bottom=True, top=True, labelbottom=False, labeltop=True)
# plt.gca().spines['top'].set_visible(False)

### surface temperature trend
originally in `GMST.ipynb`

In [ ]:
%%time
kw = {'combine':'nested', 'concat_dim':'time', 'decode_times':False}
T_rcp = xr.open_mfdataset(f'{path_prace}/rcp/atm_yrly_T_T850_U_V_2*.nc', **kw)['T'][:100,-1,:,:]
T_lr1 = xr.open_mfdataset(f'{path_prace}/lr1/atm_yrly_T_T850_U_V_2*.nc', **kw)['T'][:100,-1,:,:]
T_hq  = xr.open_mfdataset(f'{path_prace}/hq/atm_yrly_T_T850_U_V_2*.nc' , **kw)['T'][:,-1,:,:]
T_lq  = xr.open_mfdataset(f'{path_prace}/lq/atm_yrly_T_T850_U_V_2*.nc' , **kw)['T'][:51,-1,:,:]

T_rcp_diff = T_rcp.isel(time=slice(-11,-1)).mean('time')-T_rcp.isel(time=slice(0,10)).mean('time')
T_lr1_diff = T_lr1.isel(time=slice(-11,-1)).mean('time')-T_lr1.isel(time=slice(0,10)).mean('time')
T_hq_diff  = T_hq .isel(time=slice(-11,-1)).mean('time')-T_hq .isel(time=slice(0,10)).mean('time')
T_lq_diff  = T_lq .isel(time=slice(-11,-1)).mean('time')-T_lq .isel(time=slice(0,10)).mean('time')

T_rcp_trend = (xr_2D_trends(T_rcp)*100*365)
T_lr1_trend = (xr_2D_trends(T_lr1)*100*365)
T_hq_trend  = (xr_2D_trends(T_hq )*100*365)
T_lq_trend  = (xr_2D_trends(T_lq )*100*365)

In [ ]:
from xr_regression import xr_2D_trends
import xesmf as xe

In [ ]:
regridder = xe.Regridder(T_rcp, T_lr1, 'bilinear')

#### maps of SAT trends / difference

In [ ]:
(regridder(T_rcp_diff) - T_lr1_diff).plot()

In [ ]:
X1, Y1 = np.meshgrid(T_rcp.lon, T_rcp.lat)
X2, Y2 = np.meshgrid(T_lr1.lon, T_lr1.lat)

In [ ]:
def comparison_maps(xa_list, kw1, kw2, label1=None, label2=None):
    """ makse 3x2 maps comparing HIGH/LOW RCP/4xCO2 runs"""
    f = plt.figure(figsize=(6.4,5.5))
    for i in range(3):
        for j in range(2):
            xa = xa_list[j][i]
            ax = f.add_axes([.04+.48*j, .68-i*.28, .47,.27], projection=ccrs.PlateCarree())
            if j==0:  ax.text(-.07,.5, ['HIGH','LOW','HIGH-LOW'][i], transform=ax.transAxes, rotation=90, va='center')
            if i==0:  ax.set_title(['RCP', '4xCO2'][j])

            if i<2:  kw = kw1
            else:    kw = kw2

            if i==0: X, Y = X1, Y1
            else:    X, Y = X2, Y2
            im = ax.pcolormesh(X, Y, xa, **kw)
            ax.coastlines()
            
            if j==0 and i>0:
                ax = f.add_axes([.08+.48*(i-1), .08, .39,.02])
                plt.colorbar(im, cax=ax, orientation='horizontal', label=[0,label1, label2][i])
    return

xa_list = [[T_rcp_diff, T_lr1_diff, regridder(T_rcp_diff) - T_lr1_diff],
           [T_hq_diff, T_lq_diff, regridder(T_hq_diff) - T_lq_diff]]
kw1 = {'vmin':-12, 'vmax':12, 'cmap':'cmo.balance'}
kw2 = {'vmin':-5, 'vmax':5, 'cmap':'cmo.curl'}
label = 'temperature difference [K]'
comparison_maps(xa_list, kw1, kw2, label, label)

xa_list = [[T_rcp_trend, T_lr1_trend, regridder(T_rcp_trend) - T_lr1_trend],
           [T_hq_trend, T_lq_trend, regridder(T_hq_trend) - T_lq_trend]]
kw1 = {'vmin':-15, 'vmax':15, 'cmap':'cmo.balance'}
kw2 = {'vmin':-5, 'vmax':5, 'cmap':'cmo.curl'}
label = 'temperature trend [K/100yr]'
comparison_maps(xa_list, kw1, kw2, label, label)

AMOC collapses in `lq`?

In [ ]:
%%time
(xr_2D_trends(xr.open_mfdataset(f'{path_prace}/lq/atm_yrly_T_T850_U_V_2*.nc', **kw)['T'][:,-1,:,:])*100*365).plot(label='SAT trend [K/100yr]')
plt.title('SAT trend low res quadrupling')

### AMOC

In [ ]:
AMOC_rcp = xr.open_dataarray(f'{path_results}/MOC/AMOC_max_rcp.nc', decode_times=False)
AMOC_lr1 = xr.open_dataarray(f'{path_results}/MOC/AMOC_max_lr1.nc', decode_times=False)
AMOC_hq  = xr.open_dataarray(f'{path_results}/MOC/AMOC_max_hq.nc' , decode_times=False)
AMOC_lq  = xr.open_dataarray(f'{path_results}/MOC/AMOC_max_lq.nc' , decode_times=False)
plt.figure(figsize=(6.4,4))
plt.plot(AMOC_rcp.time/365, AMOC_rcp.rolling(time=12).mean(), label='HIGH RCP')
plt.plot(AMOC_lr1.time/365, AMOC_lr1.rolling(time=12).mean(), label='LOW RCP')
plt.plot(AMOC_hq .time/365, AMOC_hq .rolling(time=12).mean(), label='HIGH 4xCO2')
plt.plot(AMOC_lq .time/365, AMOC_lq .rolling(time=12).mean(), label='LOW 4xCO2')
plt.ylabel('AMOC [Sv]')
plt.xticks(np.arange(2000,2101,10))
plt.xlabel('time [years]')
plt.legend()
plt.text(.01,.01, '12 month running mean', c='grey', transform=plt.gca().transAxes)

#### meridional SAT trend

In [ ]:
T_rcp_low = regridder(T_rcp).assign_coords({'time':T_lr1.time})
T_hq_low  = regridder(T_hq) .assign_coords({'time':T_lq .time})
T_rcp_zm = (T_rcp.mean('lon')-T_rcp.isel(time=slice(0,10)).mean(['lon','time'])).T
T_lr1_zm = (T_lr1.mean('lon')-T_lr1.isel(time=slice(0,10)).mean(['lon','time'])).T
T_hq_zm  = (T_hq .mean('lon')-T_hq .isel(time=slice(0,10)).mean(['lon','time'])).T
T_lq_zm  = (T_lq .mean('lon')-T_lq .isel(time=slice(0,10)).mean(['lon','time'])).T
T_rdiff_zm = ((T_rcp_low-T_lr1).mean('lon')-(T_rcp_low-T_lr1).isel(time=slice(0,10)).mean(['lon','time'])).T
T_qdiff_zm = ((T_hq_low -T_lq ).mean('lon')-(T_hq_low -T_lq ).isel(time=slice(0,10)).mean(['lon','time'])).T

In [ ]:
%%time
#21.1 s
f, ax = plt.subplots(3, 2, figsize=(6.4,6), sharex='col', sharey=True,
                     gridspec_kw={'width_ratios':[2,1]}, constrained_layout=True)
kw1 = {'vmin':-8, 'vmax':8, 'cmap':'cmo.balance'}
kw2 = {'vmin':-5, 'vmax':5, 'cmap':'cmo.curl'}
X1, Y1 = np.meshgrid(T_rcp.time/365+2000, T_rcp.lat)
X2, Y2 = np.meshgrid(T_lr1.time/365+2000, T_lr1.lat)
X3, Y3 = np.meshgrid(T_hq .time/365+2000, T_hq .lat)
X4, Y4 = np.meshgrid(T_lq .time/365+2000, T_lq .lat)

ax[0,0].pcolormesh(X1, Y1, T_rcp_zm  , **kw1)
ax[0,1].pcolormesh(X3, Y3, T_hq_zm   , **kw1)
ax[1,0].pcolormesh(X2, Y2, T_lr1_zm  , **kw1)
ax[1,1].pcolormesh(X4, Y4, T_lq_zm   , **kw1)
ax[2,0].pcolormesh(X2, Y2, T_rdiff_zm, **kw2)
ax[2,1].pcolormesh(X4, Y4, T_qdiff_zm, **kw2)
ax[0,0].set_ylabel('HIGH')
ax[1,0].set_ylabel('LOW')
ax[2,0].set_ylabel('HIGH - LOW')
ax[0,0].set_title('RCP')
ax[0,1].set_title('4xCO2')
ax[2,0].set_xlim(2000,2100)
ax[2,1].set_xlim(2000,2050)


In [ ]:
plt.plot(T_rcp.lat, xr_linear_trend(T_rcp.mean('lon'))*365*100, ls='-' , c='C0', label='HHIGH RCP'  )
plt.plot(T_lr1.lat, xr_linear_trend(T_lr1.mean('lon'))*365*100, ls='--', c='C0', label='LOW RCP'   )
plt.plot(T_hq .lat, xr_linear_trend(T_hq .mean('lon'))*365*100, ls='-' , c='C1', label='HIGH 4xCO2')
plt.plot(T_lq .lat, xr_linear_trend(T_lq .mean('lon'))*365*100, ls='--', c='C1', label='LOW 4xCO2' )
plt.legend(ncol=2)
plt.ylabel('linear trends [K/100yr]')
plt.xlabel('latitude')

In [ ]:
plt.plot(T_rcp.lat, (T_rcp.isel(time=slice(-11,-1)).mean('time')-T_rcp.isel(time=slice(0,10)).mean('time')).mean('lon'), ls='-' , c='C0', label='HIGH RCP'  )
plt.plot(T_lr1.lat, (T_lr1.isel(time=slice(-11,-1)).mean('time')-T_lr1.isel(time=slice(0,10)).mean('time')).mean('lon'), ls='--', c='C0', label='LOW RCP'   )
plt.plot(T_hq .lat, (T_hq .isel(time=slice(-11,-1)).mean('time')-T_hq .isel(time=slice(0,10)).mean('time')).mean('lon'), ls='-' , c='C1', label='HIGH 4xCO2')
plt.plot(T_lq .lat, (T_lq .isel(time=slice(-11,-1)).mean('time')-T_lq .isel(time=slice(0,10)).mean('time')).mean('lon'), ls='--', c='C1', label='LOW 4xCO2' )
plt.legend(ncol=2)
plt.ylabel('(last decade)-(first decade) [K]')
plt.xlabel('latitude')

The LOW 4xCO2 Arctic warms dramatically despite near AMOC collapse (much steeper gradient), that could mean, atmospheric heat flux much greater, or local feedbacks much stronger.

## mean, trend, variability

## OHC patterns see `CESM_104_vs_112_OHC.ipynb`

In [ ]:
# # data to Robbin
# a = (gmst_rcp.rolling(time=12, center=True).mean()[6:12*100+6:12]).assign_coords(time=np.arange(2000,2100))
# b = (gmst_lr1.rolling(time=12, center=True).mean()[6:12*100+6:12]).assign_coords(time=np.arange(2000,2100))
# c = (gmst_hq .rolling(time=12, center=True).mean()[6:12*50 +6:12]).assign_coords(time=np.arange(2000,2050))
# d = (gmst_lq .rolling(time=12, center=True).mean()[6:12*50 +6:12]).assign_coords(time=np.arange(2000,2050))
# e = FTNT_rcp[:100].assign_coords(time=np.arange(2000,2100))
# f = FTNT_lr1[:100].assign_coords(time=np.arange(2000,2100))
# g = FTNT_hq [:50 ].assign_coords(time=np.arange(2000,2050))
# h = FTNT_lq [:50 ].assign_coords(time=np.arange(2000,2050))

# a.name = 'GMST HIGH RCP'
# b.name = 'GMST LOW RCP'
# c.name = 'GMST HIGH 4xCO2'
# d.name = 'GMST LOW 4xCO2'
# e.name = 'TOA HIGH RCP'
# f.name = 'TOA LOW RCP'
# g.name = 'TOA HIGH 4xCO2'
# h.name = 'TOA LOW 4xCO2'

# dsl = [a, b, c, d, e, f, g, h]

# xr.merge(dsl).to_netcdf(f'{path_results}/data_GMST_TOA_CESM.nc')
# ds = xr.open_dataset(f'{path_results}/data_GMST_TOA_CESM.nc', decode_times=False)

In [ ]:
f = plt.figure(figsize=(6.4,6))